# SIADS 699 Capstone: June-Aug 2023:  ETF Data

In [1]:
# Import modules
import numpy as np
import pandas as pd
import yfinance as yf
import talib as ta


In [3]:
pip install TA-lib

Note: you may need to restart the kernel to use updated packages.


# Description of ETF Tickers

In [60]:
# Create lists of relevant tickers

# US Equity Securities 

# SPY:  S&P 500
# IJH:  Mid-Cap US Equity
# IJR:  Small-cap US Equity

# US Sector ETFs

# QQQ:  Technology
# VHT:  Health Care
# XLE:  Energy
# VNQ:  Real Estate
# XLF:  Financials
# XLP:  Consumer Staples
# XLY:  Consumer Discretionary
benchmarks = ['SPY','TLT']
equity_tickers = ['SPY','XLE','XLU','XLK','XLB','XLP','XLY','XLI','XLC','XLV','XLF','XLRE']

# International Equity Securities

# VEA:  Developed Markets
# VWO:  Emerging Markets

# Fixed Income Securities

# TLT:  Long-dated Treasuries (20 yrs)
# IEF:  Intermediate-dated Treasures(7-10 yrs)
# SHY:  Short-dated Treasuries ()
# LQD:  Investment grade corporate debt
# HYG:  High Yield corporate debt

debt_tickers = ['TLT','HYBL','SJNK','STOT','SPTI']#['TLT','IEF','SHY','LQD','HYG']

all_tickers = equity_tickers + debt_tickers
print(len(all_tickers))

17


# Function to get ETF Data

In [8]:
def get_hist_ETF_data(tickers,start_date,end_date):
    
    combined_etf_data = pd.DataFrame()
    
    for i in tickers:
    
        # Get ETF data
        data = yf.download(i, start=start_date, end=end_date, interval = "1d")
        
        # Get dividend data
        
        # Get div info for specific stock
        ticks = yf.Ticker(i)
        # Get historical div payments
        ticks_hist = ticks.history(start=start_date, end=end_date, interval = "1d")['Dividends']
        # Convert into df
        ticks_hist_df = pd.DataFrame(ticks_hist)
        # Make index same as base df
        ticks_hist_df.index = data.index
        # Add new column to base df with dividend info
        data["Dividends"] = ticks_hist_df
        
        # Simple moving average
        data['MA'] = ta.SMA(data['Adj Close'],timeperiod=5)

        # Exponential moving average
        data['EMA'] = ta.EMA(data['Adj Close'],timeperiod=5)

        # Relative Strength Index (RSI)
        data['RSI'] = ta.RSI(data['Adj Close'],timeperiod=14)

        # Moving Average Convergence-Divergence (MACD)
        macd, macdsignal, macdhist = ta.MACD(data['Adj Close'], fastperiod=12, slowperiod=26, signalperiod=9)
        data['MACD'] = macd
        data['Signal'] = macdsignal
        
        # Calculate different between MACD and Signal lines
        data['MACD_minus_signal'] = data['MACD'] - data['Signal']
        
        # Take subset of needed columns
        subset_df = data[['Adj Close','Dividends','EMA','RSI','MACD','Signal','MACD_minus_signal']]
        subset_df.columns = [str(i)+'_Adj Close',str(i)+'_Dividends',str(i)+'_EMA',str(i)+'_RSI',
                             str(i)+'_MACD',str(i)+'_Signal',str(i)+'_MACD_minus_signal']
        
        # Add subset_df to combined df
        combined_etf_data = pd.concat([combined_etf_data,subset_df],axis=1)
        
    return combined_etf_data

# Test function on 60/40 Equity/Debt portfolio

In [33]:
# Set params for function
ticker_test = ['SPY','XLE','XLU','XLK','XLB','XLP','XLY','XLI','XLC','XLV','XLF','XLRE','TLT','HYBL','SJNK','SPTI']
start_date = "2007-11-01"
end_date = "2023-7-17"

In [83]:
# Run function
test_60_40 = get_hist_ETF_data(ticker_test,start_date,end_date)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [84]:
test_60_40.columns

Index(['SPY_Adj Close', 'SPY_Dividends', 'SPY_EMA', 'SPY_RSI', 'SPY_MACD',
       'SPY_Signal', 'SPY_MACD_minus_signal', 'XLE_Adj Close', 'XLE_Dividends',
       'XLE_EMA',
       ...
       'SJNK_MACD', 'SJNK_Signal', 'SJNK_MACD_minus_signal', 'SPTI_Adj Close',
       'SPTI_Dividends', 'SPTI_EMA', 'SPTI_RSI', 'SPTI_MACD', 'SPTI_Signal',
       'SPTI_MACD_minus_signal'],
      dtype='object', length=112)

Add Back Dividends to Adjusted Close

In [22]:
test_60_40['Spy_Adj Close'] = test_60_40['SPY_Adj Close']+ test_60_40['SPY_Dividends']
test_60_40['TLT_Adj Close'] = test_60_40['TLT_Adj Close']+ test_60_40['TLT_Dividends']
#'XLE','XLU','XLK','XLB','XLP','XLY','XLI','XLC','XLV','XLF','XLRE','TLT','HYBL','SJNK','STOT','SPTI'
test_60_40['XLE_Adj Close'] = test_60_40['XLE_Adj Close']+ test_60_40['XLE_Dividends']

In [71]:
ticker_quotes = set(ticker_test)^set(benchmarks) # excludes  benchmarks 


{'HYBL',
 'SJNK',
 'SPTI',
 'XLB',
 'XLC',
 'XLE',
 'XLF',
 'XLI',
 'XLK',
 'XLP',
 'XLRE',
 'XLU',
 'XLV',
 'XLY'}

In [79]:
ticker_quotes = set(ticker_test)^set(benchmarks) # excludes  benchmarks 

for j in benchmarks:
    test_60_40[(j)+'_Adj Close'] = test_60_40[(j)+'_Adj Close']+ test_60_40[(j)+'_Dividends']    
    test_60_40 = test_60_40.drop([(j)+'_Dividends'], axis=1)


for i in ticker_quotes:
    test_60_40[(i)+'_Adj Close'] = test_60_40[(i)+'_Adj Close']+ test_60_40[(i)+'_Dividends']
    test_60_40 = test_60_40.drop([(i)+'_Dividends',(i)+'_EMA',(i)+'_RSI',(i)+'_MACD',(i)+'_Signal',(i)+'_MACD_minus_signal'], axis=1)

In [80]:
test_60_40

,SPY_Adj Close,SPY_EMA,SPY_RSI,SPY_MACD,SPY_Signal,SPY_MACD_minus_signal,XLE_Adj Close,XLU_Adj Close,XLK_Adj Close,XLB_Adj Close,...,XLRE_Adj Close,TLT_Adj Close,TLT_EMA,TLT_RSI,TLT_MACD,TLT_Signal,TLT_MACD_minus_signal,HYBL_Adj Close,SJNK_Adj Close,SPTI_Adj Close
Date,,,,,,,,,,,,,,,,,,,,,
2007-11-01,111.071999,NaN,NaN,NaN,NaN,NaN,46.070751,23.679356,22.312790,29.843166,...,NaN,58.544654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.764126
2007-11-02,111.197037,NaN,NaN,NaN,NaN,NaN,46.937107,23.872980,22.360449,30.111010,...,NaN,58.336575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.663439
2007-11-05,110.351311,NaN,NaN,NaN,NaN,NaN,46.559624,24.134943,22.328674,29.758595,...,NaN,58.310921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.659626
2007-11-06,111.836884,NaN,NaN,NaN,NaN,NaN,48.112831,24.157721,22.447817,30.456379,...,NaN,58.028999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.625263
2007-11-07,108.777466,110.646939,NaN,NaN,NaN,NaN,46.132645,23.605335,21.844131,29.603518,...,NaN,58.131500,58.191730,NaN,NaN,NaN,NaN,NaN,NaN,20.648180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-10,439.660004,439.884655,60.943045,4.807296,5.332990,-0.525695,81.419998,65.110001,171.660004,81.220001,...,37.919998,99.209999,100.067279,37.603417,-0.586157,-0.204532,-0.381625,27.565001,24.480000,28.049999
2023-07-11,442.459991,440.743100,64.394079,4.767644,5.219921,-0.452277,83.209999,65.919998,172.029999,82.019997,...,38.349998,99.720001,99.951520,40.636522,-0.650608,-0.293747,-0.356861,27.639999,24.559999,28.059999
2023-07-12,446.019989,442.502063,68.236914,4.966234,5.169184,-0.202950,83.980003,66.879997,173.979996,83.099998,...,38.529999,100.830002,100.244347,46.708378,-0.605142,-0.356026,-0.249116,27.754000,24.709999,28.270000


In [81]:
# Show results
test_60_40.to_csv('test_60_40_advanced.csv')

In [82]:
test_60_40.head(200)

,SPY_Adj Close,SPY_EMA,SPY_RSI,SPY_MACD,SPY_Signal,SPY_MACD_minus_signal,XLE_Adj Close,XLU_Adj Close,XLK_Adj Close,XLB_Adj Close,...,XLRE_Adj Close,TLT_Adj Close,TLT_EMA,TLT_RSI,TLT_MACD,TLT_Signal,TLT_MACD_minus_signal,HYBL_Adj Close,SJNK_Adj Close,SPTI_Adj Close
Date,,,,,,,,,,,,,,,,,,,,,
2007-11-01,111.071999,NaN,NaN,NaN,NaN,NaN,46.070751,23.679356,22.312790,29.843166,...,NaN,58.544654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.764126
2007-11-02,111.197037,NaN,NaN,NaN,NaN,NaN,46.937107,23.872980,22.360449,30.111010,...,NaN,58.336575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.663439
2007-11-05,110.351311,NaN,NaN,NaN,NaN,NaN,46.559624,24.134943,22.328674,29.758595,...,NaN,58.310921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.659626
2007-11-06,111.836884,NaN,NaN,NaN,NaN,NaN,48.112831,24.157721,22.447817,30.456379,...,NaN,58.028999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.625263
2007-11-07,108.777466,110.646939,NaN,NaN,NaN,NaN,46.132645,23.605335,21.844131,29.603518,...,NaN,58.131500,58.191730,NaN,NaN,NaN,NaN,NaN,NaN,20.648180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-08-12,96.586929,96.368748,53.915886,0.174083,-0.336467,0.510550,44.286217,21.624889,18.851500,27.849846,...,NaN,60.963860,60.664592,55.968622,0.119689,0.052116,0.067574,NaN,NaN,21.816486
2008-08-13,96.004509,96.247335,51.861514,0.195188,-0.230136,0.425324,45.852047,21.624889,18.779455,28.452164,...,NaN,60.751720,60.693635,53.741107,0.132628,0.068218,0.064410,NaN,NaN,21.777288
2008-08-14,96.728783,96.407818,54.198688,0.267276,-0.130654,0.397930,45.009880,21.438566,18.923542,28.394806,...,NaN,61.142811,60.843360,57.128646,0.172453,0.089065,0.083387,NaN,NaN,21.828255
